In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import pandas as pd

base_dir = r"C:\Users\TanPhat\Documents\DEEPLEARNING\deepfake_detection\preprocessing\output_faces"

data = []

for label in ["fake", "real"]:
    folder_path = os.path.join(base_dir, label)
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith((".jpg", ".png", ".jpeg")):
                frame_path = os.path.join(root, file)
                data.append({
                    "frame_path": frame_path,
                    "label": label
                })

df = pd.DataFrame(data)

print(f"Tổng số ảnh: {len(df)}")
print("Phân bố labels:")
print(df["label"].value_counts())
print(df.sample(10))


In [ ]:
import cv2
import matplotlib.pyplot as plt
import random

def show_random_images(df, n=9):
    samples = df.sample(n)

    cols = int(n ** 0.5)
    rows = (n + cols - 1) // cols

    plt.figure(figsize=(cols * 3, rows * 3))

    for i, (_, row) in enumerate(samples.iterrows()):
        img = cv2.imread(row["frame_path"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.subplot(rows, cols, i + 1)
        plt.imshow(img)
        plt.title(row["label"])
        plt.axis("off")

    plt.tight_layout()
    plt.show()


# ===== xem thử =====
show_random_images(df, n=9)


In [ ]:
from pathlib import Path

df["video_id"] = df["frame_path"].apply(
    lambda x: Path(x).parent.name
)

df[["video_id", "label"]].head()

In [ ]:
video_df = (
    df[["video_id", "label"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

print(video_df["label"].value_counts())

In [ ]:
from sklearn.model_selection import train_test_split

train_videos, temp_videos = train_test_split(
    video_df,
    test_size=0.30,
    stratify=video_df["label"],
    random_state=42,
)

val_videos, test_videos = train_test_split(
    temp_videos,
    test_size=0.50,
    stratify=temp_videos["label"],
    random_state=42,
)


In [ ]:
train_df = df[df["video_id"].isin(train_videos["video_id"])]
val_df   = df[df["video_id"].isin(val_videos["video_id"])]
test_df  = df[df["video_id"].isin(test_videos["video_id"])]

In [ ]:
assert set(train_df.video_id).isdisjoint(val_df.video_id)
assert set(train_df.video_id).isdisjoint(test_df.video_id)
assert set(val_df.video_id).isdisjoint(test_df.video_id)

print("Train:\n", train_df["label"].value_counts())
print("Val:\n", val_df["label"].value_counts())
print("Test:\n", test_df["label"].value_counts())

In [ ]:
IMG_SIZE = (160, 160)
BATCH_SIZE = 16
EPOCHS = 20
LEARNING_RATE = 1e-5
FINE_TUNE_PERCENT = 0.2

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.08,
    brightness_range=[0.9, 1.1],
    fill_mode="nearest",
    shear_range=0.05,

)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col="frame_path",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=16,              
    class_mode="binary",
    shuffle=True,
)

validation_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col="frame_path",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=16,
    class_mode="binary",
    shuffle=False,
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col="frame_path",
    y_col="label",
    target_size=IMG_SIZE,
    batch_size=16,
    class_mode="binary",
    shuffle=False,
)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

y_train = train_df["label"].astype(str).map({"fake": 0, "real": 1}).values

class_weights_array = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0, 1]),
    y=y_train,
)

class_weights = {
    0: class_weights_array[0],  # fake
    1: class_weights_array[1],  # real
}

print(class_weights)

In [ ]:
# Load MobileNetV2 pretrained
base_model = MobileNetV2(
    input_shape=(*IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Fine-tune top 20% layers (kết hợp ưu điểm code cũ)
fine_tune_at = int(len(base_model.layers) * (1 - FINE_TUNE_PERCENT))
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in base_model.layers[fine_tune_at:]:
    layer.trainable = True

trainable_layers = sum([layer.trainable for layer in base_model.layers])
print(f" Base model: {len(base_model.layers)} layers")
print(f"   - Frozen layers   : {len(base_model.layers) - trainable_layers}")
print(f"   - Trainable layers: {trainable_layers} (Top {FINE_TUNE_PERCENT*100:.0f}%)")

# Custom classification head - cân bằng giữa 2 code
x = base_model.output
x = GlobalAveragePooling2D(name='global_avg_pool')(x)

# Dense layers với regularization (từ code mới)
x = Dense(256, activation='relu', kernel_regularizer=l2(1e-4), name='dense_1')(x)
x = BatchNormalization(name='bn_1')(x)
x = Dropout(0.4, name='dropout_1')(x)

x = Dense(128, activation='relu', kernel_regularizer=l2(1e-4), name='dense_2')(x)
x = BatchNormalization(name='bn_2')(x)
x = Dropout(0.3, name='dropout_2')(x)

# Output layer: Binary classification
predictions = Dense(1, activation='sigmoid', name='output')(x)

# Tạo model
model = Model(inputs=base_model.input, outputs=predictions, name='MobileNetV2_Deepfake_FineTune_V2')

In [ ]:
# Compile Model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ]
)

print("\n Model Architecture:")
model.summary()
total_params = model.count_params()
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
print(f"Total parameters    : {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Frozen parameters   : {total_params - trainable_params:,}")


In [ ]:
# callbacks
checkpoint = ModelCheckpoint(
    'best_model_finetune.keras',
    monitor='val_auc',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_auc',
    patience=5,
    restore_best_weights=True,
    verbose=1,
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_auc',
    factor=0.5,
    patience=2,
    min_lr=1e-7,
    mode='max',
    verbose=1
)


In [ ]:
# Training
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[checkpoint, early_stopping, reduce_lr],
    class_weight=class_weights,
    verbose=1,
)

In [ ]:
model.save('finetune_MobileNetV2_new.keras')

In [ ]:
# Evaluation
best_model = load_model(r'C:\Users\TanPhat\Documents\DEEPLEARNING\deepfake_detection\model_mobileNetV2\train_model\finetune_MobileNetV2_new.keras')

# Đánh giá trên Validation Set
print("\n Đánh giá trên VALIDATION SET:")
val_results = best_model.evaluate(validation_generator, verbose=0)
print(f"   Loss      : {val_results[0]:.4f}")
print(f"   Accuracy  : {val_results[1]*100:.2f}%")
print(f"   Precision : {val_results[2]*100:.2f}%")
print(f"   Recall    : {val_results[3]*100:.2f}%")
print(f"   AUC-ROC   : {val_results[4]:.4f}")

# Đánh giá trên Test Set 
print("\n Đánh giá trên TEST SET (Final Performance):")
test_results = best_model.evaluate(test_generator, verbose=0)
print(f"   Loss      : {test_results[0]:.4f}")
print(f"   Accuracy  : {test_results[1]*100:.2f}%")
print(f"   Precision : {test_results[2]*100:.2f}%")
print(f"   Recall    : {test_results[3]*100:.2f}%")
print(f"   AUC-ROC   : {test_results[4]:.4f}")


In [ ]:
# Prediction -Test Set
# Reset generator
test_generator.reset()
y_pred_prob = best_model.predict(test_generator, verbose=1).flatten()
y_pred_classes = (y_pred_prob > 0.5).astype(int)
y_true = test_generator.classes

# Classification Report
print("CLASSIFICATION REPORT (TEST SET):")
class_names = ['Real', 'Fake']
print(classification_report(y_true, y_pred_classes, target_names=class_names, digits=4))

In [ ]:
# Visualization

# Confusion Matrix - Test Set
cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Deepfake Detection (Test Set)', 
          fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)

# Thêm metrics
accuracy = (cm[0,0] + cm[1,1]) / cm.sum()
precision = cm[1,1] / (cm[1,1] + cm[0,1]) if (cm[1,1] + cm[0,1]) > 0 else 0
recall = cm[1,1] / (cm[1,1] + cm[1,0]) if (cm[1,1] + cm[1,0]) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

plt.text(0.5, -0.15, 
         f'Accuracy: {accuracy*100:.2f}% | Precision: {precision*100:.2f}% | Recall: {recall*100:.2f}% | F1: {f1:.4f}', 
         ha='center', transform=plt.gca().transAxes, fontsize=11)

plt.tight_layout()
plt.savefig('confusion_matrix_test.png', dpi=300, bbox_inches='tight')
print("Saved: confusion_matrix_test.png")
plt.show()


In [ ]:
# Roc Curve - Test Set
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Deepfake Detection (Test Set)', fontsize=16, fontweight='bold')
plt.legend(loc="lower right", fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('roc_curve_test.png', dpi=300, bbox_inches='tight')
print("Save: roc_curve_test.png")
plt.show()

In [ ]:
# Grad_Cam Đánh giá Model

import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8
    return heatmap

In [ ]:
# Visualize Grad-Cam trên 1 ảnh
def show_gradcam(img_path, model):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    heatmap = make_gradcam_heatmap(img_array, model, "out_relu")

    img = cv2.imread(img_path)
    img = cv2.resize(img, IMG_SIZE)

    heatmap = cv2.resize(heatmap, IMG_SIZE)
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)

    plt.figure(figsize=(6,6))
    plt.imshow(cv2.cvtColor(superimposed, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Grad-CAM")
    plt.show()

sample_path = test_df.sample(1)["frame_path"].values[0]
show_gradcam(sample_path,best_model)
